In [1]:
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession, functions as F

# Create a spark session
spark = (
    SparkSession.builder.appName("BNPL Project")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/29 15:53:52 WARN Utils: Your hostname, LAPTOP-VAB0S7AL resolves to a loopback address: 127.0.1.1; using 172.25.148.245 instead (on interface eth0)
22/09/29 15:53:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/29 15:53:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from etl_ext_datasets_funcs import etl_population, etl_income, join_ext_with_master

In [3]:
etl_pop = etl_population()

In [5]:
etl_pop.limit(5)

postcode,estimated_region_population_2021_sum
0200,2161
0800,7088
0801,7088
0804,2827
0810,35007


In [4]:
etl_pop.orderBy('postcode')

postcode,estimated_region_population_2021_sum
0200,2161
0800,7088
0801,7088
0804,2827
0810,35007
0811,3154
0812,18442
0813,2848
0814,3719
0815,3154


In [14]:
etl_pop.printSchema()

root
 |-- postcode: string (nullable = true)
 |-- estimated_region_population_2021_sum: long (nullable = true)



In [5]:
sdf = spark.read.parquet("../data/curated/process_data.parquet/").select('postcode', 'mean_earnings_2018-19_avg')

In [13]:
spark.read.parquet("../data/curated/process_data.parquet/").printSchema()
#all.

root
 |-- merchant_abn: long (nullable = true)
 |-- consumer_id: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- rate: double (nullable = true)
 |-- category: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- merchant_fraud_probability: double (nullable = true)
 |-- user_fraud_probability: double (nullable = true)
 |-- estimated_region_population_2021_sum: long (nullable = true)
 |-- persons_earners_2018-19_sum: double (nullable = true)
 |-- mean_earnings_2018-19_avg: double (nullable = true)
 |-- sum_earnings_2018-19_sum: double (nullable = true)
 |-- median_earnings_2018-19_avg: double (nu

In [16]:
sdf.where((sdf['postcode'] <= 860) & (sdf['postcode'] > 800)).distinct().orderBy('postcode').limit(20)

postcode,mean_earnings_2018-19_avg
801,74682.0
804,88303.0
810,69868.83333333333
811,67459.0
812,70373.0
813,44246.0
814,83448.0
815,67459.0
820,77045.90909090909
821,73384.0


In [6]:
sdf.orderBy('postcode')
# postcode still considered a string lmao
# so why wouldnt it join?

postcode,mean_earnings_2018-19_avg
200,19479.0
200,19479.0
200,19479.0
200,19479.0
200,19479.0
200,19479.0
200,19479.0
200,19479.0
200,19479.0
200,19479.0


In [11]:
sdf = sdf.withColumn('postcode', sdf['postcode'].cast('int'))

In [12]:
sdf.orderBy('postcode')

postcode
200
200
200
200
200
200
200
200
200
200


In [ ]:
# what we got -> we have proccessed_data parquet that is a string format or some shit


In [19]:
inc = etl_income()

In [20]:
pop = etl_population()

In [22]:
pop.orderBy('postcode').limit(5)

postcode,estimated_region_population_2021_sum
0200,2161
0800,7088
0801,7088
0804,2827
0810,35007


In [23]:
inc.orderBy('postcode').limit(5)

postcode,persons_earners_2018-19_sum,mean_earnings_2018-19_avg,sum_earnings_2018-19_sum,median_earnings_2018-19_avg,med_age_earners_2018-19_avg
0200,552.0,19479.0,1.0752338E7,10433.0,23.0
0800,5632.0,74682.0,4.20609031E8,57789.0,33.0
0801,5632.0,74682.0,4.20609031E8,57789.0,33.0
0804,1810.0,88303.0,1.59828824E8,71724.0,40.0
0810,21932.0,69868.83333333333,1.574969237E9,60190.166666666664,39.583333333333336


In [10]:
sdf.orderBy('postcode')

postcode,mean_earnings_2018-19_avg
200,19479.0
200,19479.0
200,19479.0
200,19479.0
200,19479.0
200,19479.0
200,19479.0
200,19479.0
200,19479.0
200,19479.0
